In [654]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.losses import MeanSquaredError
#from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import backend as K
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.python.framework.ops import disable_eager_execution
from imblearn.over_sampling import SMOTE

In [655]:
disable_eager_execution()
learning_rate = 0.01

In [656]:
# Load the already encoded data
file_path = "../data/encoded_176398_HEAD.csv"
df = pd.read_csv(file_path)
print(f"Data loaded successfully from {file_path}.\n")
print("First 5 rows of the dataset before any processing:")
print(df.head())  # Print first few rows to understand the data structure

Data loaded successfully from ../data/encoded_176398_HEAD.csv.

First 5 rows of the dataset before any processing:
              datetime  sourceID  timediff  ZAxisInPossible  ZAxisOutPossible  \
0  2023-03-27 08:14:34        10       0.0              NaN               NaN   
1  2023-03-27 08:14:49         4      15.0              NaN               NaN   
2  2023-03-27 08:14:56         5      22.0              1.0               0.0   
3  2023-03-27 08:15:08         1      34.0              1.0               0.0   
4  2023-03-27 08:15:39        12      65.0              1.0               0.0   

   YAxisDownPossible  YAxisUpPossible       PTAB  BC  S1  ...  C24  EN  SHL  \
0                NaN              NaN        NaN   0 NaN  ...  NaN NaN  NaN   
1                NaN              NaN -1127700.0   0 NaN  ...  NaN NaN  NaN   
2                1.0              1.0 -1127700.0   0 NaN  ...  NaN NaN  NaN   
3                1.0              1.0 -1127700.0   1 NaN  ...  NaN NaN  NaN   
4  

In [657]:
# Drop unnecessary columns (based on your latest specification)
columns_to_drop = ['datetime', 'SN', 'ZAxisInPossible', 'ZAxisOutPossible', 'YAxisDownPossible', 
                   'YAxisUpPossible', 'BC', 'S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S4', 
                   'S5', 'S6', 'S7', 'S8', 'S9', 'BO1', 'BO2', 'BO3', 'B1', 'B2', 'B3', 'B4', 
                   'B5', 'HE2', 'HE4', 'NE2', 'HE1', 'HE3', 'NE1', 'SHA', 'HW1', 'HW2', 'HW3', 
                   '18K', 'FA', 'TO', 'BAL', 'BAR', 'BCL', 'BCR', 'HC2', 'HC4', 'HC6', 'HC7', 
                   'NC2', 'HC1', 'HC3', 'HC5', 'NC1', 'Na', 'UFL', 'PA1', 'PA2', 'PA3', 'PA4', 
                   'PA5', 'PA6', 'SP1', 'SP2', 'SP3', 'SP4', 'SP5', 'SP6', 'SP7', 'SP8', 'BL8', 
                   'BR8', 'UFS', 'HEA', 'HEP', 'SC', 'PeH', 'PeN', 'FS', 'FL', 'BY1', 'BY2', 
                   'BY3', 'BL', 'BR', 'HE', 'BL4', 'BR4', 'BL1', 'BR1', 'BL2', 'BR2', 'L7', 
                   'L4', 'H2L', 'N2L', 'H1U', 'N1U', 'He1', 'He2', 'TR1', 'TR2', 'TR3', 'TR4', 
                   'TR5', 'TR6', 'MR', 'ML', 'BL5', 'BR5', 'C24', 'EN', 'SHL', 'SHS','BodyPart_from', 
                   'BodyPart_to', 'PatientID_from', 'PatientID_to']
df = df.drop(columns=columns_to_drop)
print("Dropped unnecessary columns.")
print("Remaining columns:", df.columns.tolist())
print(df.head())

Dropped unnecessary columns.
Remaining columns: ['sourceID', 'timediff', 'PTAB', 'BodyGroup_from', 'BodyGroup_to']
   sourceID  timediff       PTAB  BodyGroup_from  BodyGroup_to
0        10       0.0        NaN               1             4
1         4      15.0 -1127700.0               1             4
2         5      22.0 -1127700.0               1             4
3         1      34.0 -1127700.0               1             4
4        12      65.0 -1127700.0               1             4


In [658]:
# Replace NaN values with 0 in the DataFrame
df.fillna(0, inplace=True)
print("NaN values replaced with 0.")
print(df.head())  # Display a sample of the updated data

NaN values replaced with 0.
   sourceID  timediff       PTAB  BodyGroup_from  BodyGroup_to
0        10       0.0        0.0               1             4
1         4      15.0 -1127700.0               1             4
2         5      22.0 -1127700.0               1             4
3         1      34.0 -1127700.0               1             4
4        12      65.0 -1127700.0               1             4


In [659]:
# Separate features and target (sourceID)
X = df.drop(columns=['sourceID'])
y_sourceid = df['sourceID']

In [660]:
# One-hot encode 'sourceID' if needed
encoder = OneHotEncoder(sparse=False)
y_sourceid_encoded = encoder.fit_transform(y_sourceid.values.reshape(-1, 1))
original_sourceids = encoder.categories_[0]  # Save the mapping of one-hot encoding to original sourceIDs

C:\Users\z004uyxr\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [661]:
# Print the first few rows of y_sourceid_encoded to confirm one-hot encoding worked
print("\nFirst few one-hot encoded 'sourceID' values:")
print(y_sourceid_encoded[:5])
print(X_scaled.shape)
print("Shape of one-hot encoded 'sourceID':", y_sourceid_encoded.shape)


First few one-hot encoded 'sourceID' values:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
(4501, 4)
Shape of one-hot encoded 'sourceID': (4501, 12)


In [662]:
# Scale the feature data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("\nScaled features shape:", X_scaled.shape)
print("First 5 rows of scaled data:")
print(X_scaled[:5])  # Print the first 5 rows after scaling


Scaled features shape: (4501, 4)
First 5 rows of scaled data:
[[0.         0.99947446 0.         0.27272727]
 [0.00119847 0.43504592 0.         0.27272727]
 [0.00175775 0.43504592 0.         0.27272727]
 [0.00271652 0.43504592 0.         0.27272727]
 [0.00519335 0.43504592 0.         0.27272727]]


In [663]:
# Define the VAE architecture
input_dim = X_scaled.shape[1]
latent_dim = 2  # Latent space dimension
# Check the shape of X_scaled
print(X_scaled.shape)

(4501, 4)


In [664]:
# Encoder
inputs = Input(shape=(input_dim,))
h = Dense(128, activation='relu')(inputs) 
h = Dense(64, activation='relu')(h) 
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [665]:
# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

In [666]:
# Decoder
decoder_h = Dense(64, activation='relu')
decoder_mean = Dense(128, activation='relu')
outputs = Dense(12, activation='softmax')(decoder_mean(decoder_h(z)))

In [667]:
# VAE model
vae = Model(inputs, outputs)

In [668]:
# Define VAE loss function
def vae_loss(x, x_decoded_mean):
    reconstruction_loss = tf.reduce_mean(
        tf.keras.losses.categorical_crossentropy(x, x_decoded_mean)
    )
    kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
    return reconstruction_loss + kl_loss

In [669]:
# Ensure the output layer matches the input
outputs = Dense(X_scaled.shape[1], activation='sigmoid')(decoder_mean(decoder_h(z)))

In [670]:
# Use the legacy optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)

# Compile model
vae.compile(optimizer=optimizer, loss=vae_loss)


In [671]:
# Fit the model
print("\nStarting model training...")
# Fit the model with one-hot encoded labels
history = vae.fit(X_scaled, y_sourceid_encoded, epochs=100, batch_size=32)
print("VAE model training completed.")


Starting model training...
Train on 4501 samples
Epoch 1/100
4501/4501 [==============================] - 1s 227us/sample - loss: 2.0183
Epoch 2/100
4501/4501 [==============================] - 0s 52us/sample - loss: 1.9779
Epoch 3/100
4501/4501 [==============================] - 0s 50us/sample - loss: 1.9795
Epoch 4/100
4501/4501 [==============================] - 0s 49us/sample - loss: 1.9749
Epoch 5/100
4501/4501 [==============================] - 0s 53us/sample - loss: 1.9737
Epoch 6/100
4501/4501 [==============================] - 0s 52us/sample - loss: 1.9759
Epoch 7/100
4501/4501 [==============================] - 0s 54us/sample - loss: 1.9728
Epoch 8/100
4501/4501 [==============================] - 0s 51us/sample - loss: 1.9727
Epoch 9/100
4501/4501 [==============================] - 0s 54us/sample - loss: 1.9748
Epoch 10/100
4501/4501 [==============================] - 0s 49us/sample - loss: 1.9709
Epoch 11/100
4501/4501 [==============================] - 0s 51us/sample - los

In [672]:
print("Class distribution in sourceID:")
print(y_sourceid.value_counts())

Class distribution in sourceID:
sourceID
5     1208
4     1012
1      698
12     335
8      331
10     329
9      329
7      125
2      115
3        9
6        6
11       4
Name: count, dtype: int64


In [673]:
# Predict on the training data
predicted_sourceids = vae.predict(X_scaled)

# Convert predictions to one-hot encoded format
# Using softmax output, we'll simply threshold at 0.5 for binary (or interpret probabilities directly for multi-class)
predicted_sourceids_onehot = (predicted_sourceids == predicted_sourceids.max(axis=1, keepdims=True)).astype(float)

# Check shape and print first few rows
print("\nShape of predicted one-hot encoded 'sourceID' values:", predicted_sourceids_onehot.shape)
print("First few predicted one-hot encoded 'sourceID' values:")
print(predicted_sourceids_onehot[:16])

C:\Users\z004uyxr\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,



Shape of predicted one-hot encoded 'sourceID' values: (4501, 12)
First few predicted one-hot encoded 'sourceID' values:
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [674]:
# Convert predicted probabilities back to class indices
predicted_classes = np.argmax(predicted_sourceids_onehot, axis=1)

# Create a mapping back to original sourceIDs
sourceID_mapping = encoder.inverse_transform(predicted_sourceids_onehot)

# Count predicted classes
unique, counts = np.unique(predicted_classes, return_counts=True)
predicted_class_distribution = dict(zip(unique, counts))

# Print the distribution of predicted classes
print("\nPredicted class distribution:")
print(predicted_class_distribution)


Predicted class distribution:
{4: 4501}


In [675]:
# Function to map one-hot encoded predictions back to sourceIDs
def map_onehot_to_sourceid(onehot_predictions, encoding_legend):
    sourceids = []
    for prediction in onehot_predictions:
        index = np.argmax(prediction)  # Find the index of the highest value
        sourceid = encoding_legend[index + 1]  # Map back using the legend (1-based index)
        sourceids.append(sourceid)
    return sourceids

# Encoding legend mapping
encoding_legend = {
    1: 'MRI_CCS_11',
    2: 'MRI_EXU_95',
    3: 'MRI_FRR_18',
    4: 'MRI_FRR_257',
    5: 'MRI_FRR_264',
    6: 'MRI_FRR_3',
    7: 'MRI_FRR_34',
    8: 'MRI_MPT_1005',
    9: 'MRI_MSR_100',
    10: 'MRI_MSR_104',
    11: 'MRI_MSR_21',
    12: 'MRI_MSR_34'
}

# Map predicted one-hot encodings to original sourceIDs
predicted_sourceids_final = map_onehot_to_sourceid(predicted_sourceids_onehot, encoding_legend)

# Print the final predicted sourceIDs
print("\nFirst few predicted sourceIDs:")
print(predicted_sourceids_final[:16])


First few predicted sourceIDs:
['MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264', 'MRI_FRR_264']
